In [ ]:
# imports

import cv2
import numpy as np

# Point operations (I.4):  

In [ ]:
# changing exposure/brightness {brightening (1.5)}


In [ ]:
# contrast enhancement (I.6)


In [ ]:
# enhancing saturation

# converting to black and white

In [ ]:
# Introducing random salt and pepper noise


# Filtering using convolution (I.7, 15)

In [ ]:
# Gaussian Blur (example of weighted averaging filter (I.12))

# Load the image
image = cv2.imread('input_image.png')

# Apply Gaussian blur with a 5x5 kernel and default sigma values
blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

# Display the original and blurred images
cv2.imshow('Original Image', image)
cv2.imshow('Blurred Image', blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Sharpen the image (I.55-59)


# Other (from linear algebra review):

In [ ]:
# Fixed/Random rotation

In [ ]:
# Shearing; typically padded with 0s (I.9), change in basis

In [ ]:
# Flipping image (reflection) 